In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(
    temperature=0,
    model="llama3-8b-8192",
    # api_key="" # Optional if not set as an environment variable
)

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", human),
    ]
)

chain = prompt | chat
chain.invoke(
    {
        "text": "Explain the importance of low latency for LLMs.",
    }
)

AIMessage(content="Low latency is crucial for Large Language Models (LLMs) because it enables them to process and respond to user input in a timely and efficient manner. Here are some reasons why low latency is important for LLMs:\n\n1. **Improved User Experience**: Low latency ensures that users can interact with the LLM quickly and seamlessly, without having to wait for responses. This is particularly important for applications like chatbots, virtual assistants, and language translation tools, where speed and responsiveness are critical.\n2. **Increased Accuracy**: LLMs rely on complex algorithms and computations to generate responses. Low latency allows these models to process and analyze user input more quickly, which can lead to more accurate and relevant responses.\n3. **Reduced Errors**: High latency can lead to errors and misunderstandings, as the LLM may not have enough time to process the user's input correctly. Low latency reduces the likelihood of errors and ensures that th

In [2]:
from typing import Optional

from langchain_core.tools import tool


@tool
def get_current_weather(location: str, unit: Optional[str]):
    """Get the current weather in a given location"""
    return "Cloudy with a chance of rain."


tool_model = chat.bind_tools(
    [get_current_weather],
    tool_choice="auto",
)

res = tool_model.invoke("What is the weather like in San Francisco and Tokyo?")

res.tool_calls

[{'name': 'get_current_weather',
  'args': {'location': 'San Francisco', 'unit': 'metric'},
  'id': 'call_mtat',
  'type': 'tool_call'},
 {'name': 'get_current_weather',
  'args': {'location': 'Tokyo', 'unit': 'metric'},
  'id': 'call_2b5t',
  'type': 'tool_call'}]

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = chat.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why did the cat join a band?', punchline='Because it wanted to be the purr-cussionist!', rating=8)

In [4]:
chat = ChatGroq(
    temperature=0,
    model="llama3-8b-8192",
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Write a Limerick about {topic}"),
    ]
)
chain = prompt | chat
await chain.ainvoke(
    {
        "topic": "The Sun",
    }
)

AIMessage(content='There once was a sun in the sky,\nWhose warmth and bright light did not die,\nIt shone with great might,\nAnd lit up the night,\nAnd brought joy to all who did fly.', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 18, 'total_tokens': 60, 'completion_time': 0.032771902, 'prompt_time': 0.004121825, 'queue_time': None, 'total_time': 0.036893727}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-13c41274-48ae-4b66-b899-ee6edbb62c1b-0', usage_metadata={'input_tokens': 18, 'output_tokens': 42, 'total_tokens': 60})

In [5]:
chat = ChatGroq(
    temperature=0,
    model="llama3-8b-8192",
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "Write a haiku about {topic}",
        )
    ]
)
chain = prompt | chat
for chunk in chain.stream(
    {
        "topic": "The Moon",
    }
):
    print(
        chunk.content,
        end="",
        flush=True,
    )

Silvery glow bright
Moon's gentle light on my facevery glow bright
Moon's gentle light on my face
Peaceful night's delight

In [6]:
chat = ChatGroq(
    model="llama3-8b-8192",
    model_kwargs={
        "response_format": {
            "type": "json_object",
        }
    },
)

system = """
You are a helpful assistant.
Always respond with a JSON object with two string keys: "response" and "followup_question".
"""
human = "{question}"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", human),
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "question": "what bear is best?",
    }
)

AIMessage(content='{\n"response": "The best bear is a matter of personal opinion, but some popular species include the polar bear, brown bear, and black bear. Each has its own unique characteristics and adaptations that make it well-suited to its environment.",\n"followup_question": "What do you think makes a bear \'best\'? Is it its size, strength, or something else?"\n}', response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 50, 'total_tokens': 128, 'completion_time': 0.061320272, 'prompt_time': 0.013660193, 'queue_time': None, 'total_time': 0.074980465}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_33d61fdfc3', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0db1885-7cab-4cd4-929b-bec530c383b3-0', usage_metadata={'input_tokens': 50, 'output_tokens': 78, 'total_tokens': 128})